#### Creating utility file

In [5]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string


def col_header_val(df, table_config):
    """ replace whitespaces in the column  and standardized column names """
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]', '_', regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(), table_config['columns']))
    expected_col.sort()
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file", mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded", missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


#### Creating yaml file

In [19]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: postcovid_reviews
table_name: edsurv
inbound_delimeter: ","
outbound_delimeter: "|"
skip_leading_rows: 1
columns:
    -business_id
    -name
    -address
    -state_
    -city
    -postal_code
    -latitude
    -longitude
    -stars
    -review_count
    -is_open
    -categories
    -hours
    -review_id
    -user_id
    -customer_stars
    -useful
    -funny
    -cool
    -text_
    -date_

Overwriting file.yaml


#### Reading data with pandas

In [11]:
import pandas as pd
df=pd.read_csv("data/postcovid_reviews.csv")
df.head()

,business_id,name,address,state_,city,postal_code,latitude,longitude,stars,review_count,...,categories,hours,review_id,user_id,customer_stars,useful,funny,cool,text_,date_
0,2WRCcQATOe_Em0k61T6kvQ,Yook Korean Grilled BBQ & Bistro,2408 Nanaimo Street,BC,Vancouver,V5N 5E4,49.262732,-123.056371,4.0,74,...,"Restaurants, Barbeque, Korean","{'Monday': '0:0-0:0', 'Wednesday': '11:30-22:0...",t2I2CLHd7MpULwwqjPv0vA,iBRhL3g62aQzZ7nrJc-KCg,1,0,0,0,Decent food horrible service. I would give thi...,2020-02-15 21:09:17
1,oug5bLTWP_YTtj1C3_X6Xw,Gourdough's Public House,2700 S Lamar Blvd,TX,Austin,78704,30.245382,-97.780437,4.0,1534,...,"Karaoke, Pubs, Comfort Food, Bars, Gastropubs,...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-0:0', '...",gcJK-cQHFrrP1wnRUsQ9Jw,qcpg7m7V-5XgHZzdk_f_hQ,4,1,0,0,I've been wanting to try Gourdough's for a whi...,2020-09-30 01:17:12
2,imUlMANF0tNDxWOXdxfP9w,Donut Express & Cakes,258 Main St,MA,Medfield,02052,42.193721,-71.290251,5.0,42,...,"Food, Bakeries, Custom Cakes, Donuts, Restaurants","{'Monday': '5:0-13:0', 'Tuesday': '5:0-13:0', ...",wvtTSr4GGGnNhfnj7K1R6g,os_bbAergdkI_OiYyLrlrg,2,0,0,0,The last custom cake bought in October 2020 wa...,2020-12-24 18:11:13
3,mP1EdIafQKMuOm9O4PzAfA,Barcelona Wine Bar South End,525 Tremont St,MA,Boston,02116,42.344936,-71.070544,4.5,1097,...,"Spanish, Bars, Tapas Bars, Wine Bars, Tapas/Sm...","{'Monday': '12:0-21:0', 'Tuesday': '12:0-21:0'...",twUhcgH6ODfMv7ZXEQMX6A,clEAaBNgrnS7PofAonfkJA,4,1,0,0,I came here with a group of 4 on a Saturday ni...,2020-02-19 00:23:46
4,4UjU7F_EX3lgUtkzN8Bbrw,Shan-A-Punjab,500 Harvard St,MA,Brookline,02446,42.348017,-71.129557,4.0,426,...,"Nightlife, Indian, Vegetarian, Cocktail Bars, ...","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:...",vPFrlWYmxdUxjXS2JQquHQ,evqTtztHLyPSy3n0KcrJYg,1,1,0,1,Ate the lunch buffet at Shan-a-Punjab many tim...,2020-01-30 23:23:16


In [12]:
df.columns

Index(['business_id', 'name', 'address', 'state_', 'city', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'categories', 'hours', 'review_id', 'user_id', 'customer_stars',
       'useful', 'funny', 'cool', 'text_', 'date_'],
      dtype='object')

#### Reading config file

In [20]:
import testutility as util
df_config= util.read_config_file("file.yaml")
df_config

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'postcovid_reviews',
 'table_name': 'edsurv',
 'inbound_delimeter': ',',
 'outbound_delimeter': '|',
 'skip_leading_rows': 1,
 'columns': '-business_id -name -address -state_ -city -postal_code -latitude -longitude -stars -review_count -is_open -categories -hours -review_id -user_id -customer_stars -useful -funny -cool -text_ -date_'}

In [23]:
import pandas as pd 
file_type=df_config['file_type']
source_file="data/"+  df_config['file_name'] + f'.{file_type}'

print(source_file)

df2=pd.read_csv(source_file,df_config['inbound_delimeter'])
print("New dataset:", df2)

data/postcovid_reviews.csv
New dataset:                    business_id                              name  \
0       2WRCcQATOe_Em0k61T6kvQ  Yook Korean Grilled BBQ & Bistro   
1       oug5bLTWP_YTtj1C3_X6Xw          Gourdough's Public House   
2       imUlMANF0tNDxWOXdxfP9w             Donut Express & Cakes   
3       mP1EdIafQKMuOm9O4PzAfA      Barcelona Wine Bar South End   
4       4UjU7F_EX3lgUtkzN8Bbrw                     Shan-A-Punjab   
...                        ...                               ...   
400290  Q78fYV6B6P6GmX07YVgi4g             Maudie’s Hill Country   
400291  RmRAhZC0FcxlwW0nEZv1GQ        Texican Cafe - South Lamar   
400292  P5x-GukFEq4UeSOl6GJfAw    Bubbalou's Bodacious Bar B Que   
400293  cgNDiWCaSlqqxx1A6r65bA                        Hop N Cork   
400294  dmkDZKPsK8lmwFuLiFQ0Zw                       Planted PDX   

                            address state_         city postal_code  \
0               2408 Nanaimo Street     BC    Vancouver     V5N 5E4   
1

#### Validate the header of file

In [24]:
util.col_header_val(df2,df_config)

column name and column length validation failed
Following File columns are not in the YAML file ['longitude', 'address', 'funny', 'business_id', 'postal_code', 'review_id', 'latitude', 'state', 'user_id', 'customer_stars', 'categories', 'review_count', 'is_open', 'date', 'stars', 'useful', 'city', 'cool', 'text', 'name', 'hours']
Following YAML columns are not in the file uploaded ['n', 'i', 'a', 't', 'w', 'm', '_', 'd', 'l', 'u', 'p', 's', 'b', 'f', 'v', 'c', 'r', 'g', ' ', 'x', '-', 'y', 'o', 'e', 'h']


0

In [25]:
if util.col_header_val(df2,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['longitude', 'address', 'funny', 'business_id', 'postal_code', 'review_id', 'latitude', 'state', 'user_id', 'customer_stars', 'categories', 'review_count', 'is_open', 'date', 'stars', 'useful', 'city', 'cool', 'text', 'name', 'hours']
Following YAML columns are not in the file uploaded ['n', 'i', 'a', 't', 'w', 'm', '_', 'd', 'l', 'u', 'p', 's', 'b', 'f', 'v', 'c', 'r', 'g', ' ', 'x', '-', 'y', 'o', 'e', 'h']
validation failed


### Summary
<p>
    Total number of rows: 400294 <br>
    Total number of columns: 21 <br>
    File size: 2+ GB
<p>